# amazon_india_2015

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np


file_path = Path("../data/amazon_india_complete_2015_2025.csv")
amazon_2015_2025 = pd.read_csv(file_path)
amazon_2015_2025.head()

,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,is_festival_sale,festival_name,customer_rating,return_status,order_month,order_year,order_quarter,product_weight_kg,is_prime_eligible,product_rating
0,TXN_2023_00063013,2023-07-23,CUST_2023_00018393,PROD_000454,Vivo Y95 64GB Black,Electronics,Smartphones,Vivo,27340.84,21.57,...,False,NaN,4.0,Delivered,7,2023,3,0.20,True,3.5
1,TXN_2021_00064486,20-07-2021,CUST_2015_00002865,PROD_000579,Realme Realme 3 128GB Black,Electronics,Smartphones,Realme,32907.49,0.00,...,False,NaN,5/5,Delivered,7,2021,3,0.21,False,4.5
2,TXN_2017_00065617,2017-11-16,CUST_2016_00004057,PROD_000295,Vivo V7 32GB Blue,Electronics,Smartphones,Vivo,"47,052.18",21.91,...,False,NaN,5.0,Delivered,11,2017,4,0.24,True,4.3
3,TXN_2020_00054393,2020-05-04,CUST_2020_00014574,PROD_001654,Alienware Pavilion 4GB RAM Silver,Electronics,Laptops,Alienware,238725.44,59.60,...,True,Summer Sale,5.0 stars,Delivered,5,2020,2,1.85,Yes,3.6
4,TXN_2018_00071646,2018-10-09,CUST_2018_00006275,PROD_000095,Motorola Moto X Play 16GB White,Electronics,Smartphones,Motorola,25970.76,0.00,...,False,NaN,4.0,Delivered,10,2018,4,0.16,False,3.7


In [2]:
amazon_2015_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127609 entries, 0 to 1127608
Data columns (total 34 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   transaction_id          1127609 non-null  object 
 1   order_date              1127609 non-null  object 
 2   customer_id             1127609 non-null  object 
 3   product_id              1127609 non-null  object 
 4   product_name            1127609 non-null  object 
 5   category                1127609 non-null  object 
 6   subcategory             1127609 non-null  object 
 7   brand                   1127609 non-null  object 
 8   original_price_inr      1127609 non-null  object 
 9   discount_percent        1127609 non-null  float64
 10  discounted_price_inr    1127609 non-null  float64
 11  quantity                1127609 non-null  int64  
 12  subtotal_inr            1127609 non-null  float64
 13  delivery_charges        1037408 non-null  float64
 14  fi

In [3]:
amazon_2015_2025.isnull().sum()

transaction_id                 0
order_date                     0
customer_id                    0
product_id                     0
product_name                   0
category                       0
subcategory                    0
brand                          0
original_price_inr             0
discount_percent               0
discounted_price_inr           0
quantity                       0
subtotal_inr                   0
delivery_charges           90201
final_amount_inr               0
customer_city                  0
customer_state                 0
customer_tier                  0
customer_spending_tier         0
customer_age_group        135315
payment_method                 0
delivery_days                  0
delivery_type                  0
is_prime_member                0
is_festival_sale               0
festival_name             777736
customer_rating           341696
return_status                  0
order_month                    0
order_year                     0
order_quar

In [4]:
amazon_2015_2025.columns

Index(['transaction_id', 'order_date', 'customer_id', 'product_id',
       'product_name', 'category', 'subcategory', 'brand',
       'original_price_inr', 'discount_percent', 'discounted_price_inr',
       'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr',
       'customer_city', 'customer_state', 'customer_tier',
       'customer_spending_tier', 'customer_age_group', 'payment_method',
       'delivery_days', 'delivery_type', 'is_prime_member', 'is_festival_sale',
       'festival_name', 'customer_rating', 'return_status', 'order_month',
       'order_year', 'order_quarter', 'product_weight_kg', 'is_prime_eligible',
       'product_rating'],
      dtype='object')

In [5]:
amazon_2015_2025['order_date'] = pd.to_datetime(
    amazon_2015_2025['order_date'], 
    errors='coerce',     # turn bad ones into NaT
    dayfirst=True,       # needed for DD-MM-YYYY and DD/MM/YYYY
    format="mixed"       # <-- NEW in Pandas 2.0, handles mixed styles
)


In [6]:
amazon_2015_2025['original_price_inr'] = (
    amazon_2015_2025['original_price_inr']
    .str.replace('₹', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.replace('Rs', '', regex=False)
)

amazon_2015_2025['original_price_inr'] = pd.to_numeric(
    amazon_2015_2025['original_price_inr'], errors='coerce'
)


In [7]:
category_map = {
    'electronics': 'Electronics',
    'electronic': 'Electronics',
    'electronicss': 'Electronics',
    'ELECTRONICS': 'Electronics',
    'electronics & accessories': 'Electronics & Accessories'
}
amazon_2015_2025['category'] = (
    amazon_2015_2025['category']
    .str.lower()
    .str.strip()
    .replace(category_map)
    .str.title()
)


In [8]:
amazon_2015_2025['delivery_charges'] = amazon_2015_2025['delivery_charges'].fillna(0)

In [9]:
city_map = {
    # Bengaluru
    'bangalore': 'Bangalore',
    'bengalore': 'Bangalore',
    'BANGALORE': 'Bangalore',
    'banglore': 'Bangalore',
    'bengaluru': 'Bangalore',

    # Mumbai
    'mumbai': 'Mumbai',
    'mumba': 'Mumbai',
    'bombay': 'Mumbai',
    'Mumbai ': 'Mumbai',
    'Bombay': 'Mumbai',
    'MUMBAI': 'Mumbai',

    # Delhi
    'delhi': 'Delhi',
    'new delhi': 'Delhi',
    'delhi ncr': 'Delhi',
    'DELHI': 'Delhi',

    # Chennai
    'chennai': 'Chennai',
    'chenai': 'Chennai',
    'madras': 'Chennai',
    'CHENNAI': 'Chennai',
    'Madras': 'Chennai',
    'Chennai ': 'Chennai',

    # Kolkata
    'KOLKATA': 'Kolkata',
    'kolkata': 'Kolkata',
    'calcutta': 'Kolkata'

}
amazon_2015_2025['customer_city'] = (
    amazon_2015_2025['customer_city']
    .str.lower()
    .str.strip()
    .replace(city_map)
    .str.title()
)


In [10]:
amazon_2015_2025['customer_age_group'] = amazon_2015_2025['customer_age_group'].fillna('Unknown')

In [11]:
payment_map = {
    # UPI
    'upi': 'UPI',
    'phonepe': 'UPI',
    'googlepay': 'UPI',
    'gpay': 'UPI',

    # Credit Card
    'credit card': 'Credit Card',
    'cc': 'Credit Card',
    'credit_card': 'Credit Card',

    # Debit Card
    'debit card': 'Debit Card',
    'dc': 'Debit Card',

    # COD
    'cash on delivery': 'COD',
    'cod': 'COD',
    'c.o.d': 'COD',

    # Net Banking
    'net banking': 'Net Banking',
    'netbanking': 'Net Banking',

    # Wallet
    'wallet': 'Wallet',

    # Buy Now Pay Later
    'bnpl': 'BNPL'
}
amazon_2015_2025['payment_method'] = (
    amazon_2015_2025['payment_method']
    .str.lower()
    .str.strip()
    .replace(payment_map)
)


In [12]:
amazon_2015_2025['delivery_days'].isnull().sum()

np.int64(0)

In [13]:
amazon_2015_2025['delivery_days'] = amazon_2015_2025['delivery_days'].replace({
    'Same Day': 0,
    '1-2 days': 2,
    'Express': 1,
    "-1": None,
})

amazon_2015_2025['delivery_days'] = pd.to_numeric(amazon_2015_2025['delivery_days'], errors='coerce')

amazon_2015_2025.loc[
    (amazon_2015_2025['delivery_days'] < 0) | (amazon_2015_2025['delivery_days'] > 30),
    'delivery_days'
] = None


In [14]:
bool_map = {
    'yes': True, 'y': True, '1': True, 'true': True,
    'no': False, 'n': False, '0': False, 'false': False
}

bool_cols = ['is_prime_member', 'is_prime_eligible', 'is_festival_sale']

for col in bool_cols:
    amazon_2015_2025[col] = (
        amazon_2015_2025[col]
        .astype(str)
        .str.lower()
        .map(bool_map)
    )


In [15]:
amazon_2015_2025['festival_name'] = amazon_2015_2025['festival_name'].fillna('No Festival')

In [16]:
amazon_2015_2025['customer_rating'] = (
    amazon_2015_2025['customer_rating']
    .astype(str)
    .str.lower()
    .str.strip()
    .str.replace('stars', '', regex=False)
    .str.replace('star', '', regex=False)
)


In [17]:
def convert_fraction_rating(x):
    if '/' in x:
        try:
            num, den = x.split('/')
            return (float(num) / float(den)) * 5
        except:
            return np.nan
    return x


In [18]:
amazon_2015_2025['customer_rating'] = (
    amazon_2015_2025['customer_rating']
    .apply(convert_fraction_rating)
)


In [19]:
amazon_2015_2025['customer_rating'] = pd.to_numeric(
    amazon_2015_2025['customer_rating'],
    errors='coerce'
)


In [20]:
len(amazon_2015_2025)


1127609

In [21]:
missing_pct = (
    amazon_2015_2025['customer_rating'].isna().mean() * 100
)
print(f"Missing Ratings: {missing_pct:.2f}%")


Missing Ratings: 30.30%


In [22]:
amazon_2015_2025['customer_rating'] = (
    amazon_2015_2025
    .groupby('category')['customer_rating']
    .transform(lambda x: x.fillna(x.median()))
)


In [23]:
amazon_2015_2025.shape

(1127609, 34)

In [41]:
amazon_2015_2025['transaction_id'].nunique()

1127609

In [25]:
amazon_2015_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127609 entries, 0 to 1127608
Data columns (total 34 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   transaction_id          1127609 non-null  object        
 1   order_date              1127609 non-null  datetime64[ns]
 2   customer_id             1127609 non-null  object        
 3   product_id              1127609 non-null  object        
 4   product_name            1127609 non-null  object        
 5   category                1127609 non-null  object        
 6   subcategory             1127609 non-null  object        
 7   brand                   1127609 non-null  object        
 8   original_price_inr      1127609 non-null  float64       
 9   discount_percent        1127609 non-null  float64       
 10  discounted_price_inr    1127609 non-null  float64       
 11  quantity                1127609 non-null  int64         
 12  subtotal_inr  

In [26]:
amazon_2015_2025.to_csv("amazon_india_cleaned_2015_2025.csv", index=False)

#check cleaned data set

In [27]:
amazon_2015_2025.dtypes

transaction_id                    object
order_date                datetime64[ns]
customer_id                       object
product_id                        object
product_name                      object
category                          object
subcategory                       object
brand                             object
original_price_inr               float64
discount_percent                 float64
discounted_price_inr             float64
quantity                           int64
subtotal_inr                     float64
delivery_charges                 float64
final_amount_inr                 float64
customer_city                     object
customer_state                    object
customer_tier                     object
customer_spending_tier            object
customer_age_group                object
payment_method                    object
delivery_days                    float64
delivery_type                     object
is_prime_member                     bool
is_festival_sale

In [28]:
amazon_2015_2025.isna().sum()
amazon_2015_2025.isna().mean() * 100

transaction_id            0.000000
order_date                0.000000
customer_id               0.000000
product_id                0.000000
product_name              0.000000
category                  0.000000
subcategory               0.000000
brand                     0.000000
original_price_inr        0.000000
discount_percent          0.000000
discounted_price_inr      0.000000
quantity                  0.000000
subtotal_inr              0.000000
delivery_charges          0.000000
final_amount_inr          0.000000
customer_city             0.000000
customer_state            0.000000
customer_tier             0.000000
customer_spending_tier    0.000000
customer_age_group        0.000000
payment_method            0.000000
delivery_days             0.606239
delivery_type             0.000000
is_prime_member           0.000000
is_festival_sale          0.000000
festival_name             0.000000
customer_rating           0.000000
return_status             0.000000
order_month         

In [29]:
amazon_2015_2025.duplicated().sum()
amazon_2015_2025.duplicated(subset=['transaction_id']).sum()

np.int64(0)

In [30]:
amazon_2015_2025[['quantity', 'discount_percent', 'final_amount_inr']].describe()

,quantity,discount_percent,final_amount_inr
count,1.127609e+06,1.127609e+06,1.127609e+06
mean,1.250063e+00,1.742035e+01,6.818740e+04
std,5.365034e-01,2.055377e+01,6.893407e+04
min,1.000000e+00,0.000000e+00,3.443300e+02
25%,1.000000e+00,0.000000e+00,2.521799e+04
50%,1.000000e+00,1.037000e+01,4.473182e+04
75%,1.000000e+00,2.841000e+01,8.852179e+04
max,3.000000e+00,7.000000e+01,1.262114e+06


In [31]:
# Discount should not exceed 100%
amazon_2015_2025[amazon_2015_2025['discount_percent'] > 100]

# Quantity must be positive
amazon_2015_2025[amazon_2015_2025['quantity'] <= 0]

# Final amount should be >= 0
amazon_2015_2025[amazon_2015_2025['final_amount_inr'] < 0]

,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,is_festival_sale,festival_name,customer_rating,return_status,order_month,order_year,order_quarter,product_weight_kg,is_prime_eligible,product_rating


In [32]:
amazon_2015_2025['category'].value_counts()
  

category
Electronics                  1127391
Electronics & Accessories        218
Name: count, dtype: int64

In [33]:
amazon_2015_2025['payment_method'].value_counts()


payment_method
UPI            384228
COD            322831
Credit Card    172261
Debit Card     140202
Net Banking     64971
Wallet          22821
BNPL            20295
Name: count, dtype: int64

In [34]:
amazon_2015_2025['customer_state'].value_counts().head(10)

customer_state
Maharashtra      246558
Delhi            123469
Uttar Pradesh    118403
Gujarat          112339
Tamil Nadu       111448
Karnataka        102523
West Bengal       66867
Telangana         44573
Rajasthan         40453
Punjab            38466
Name: count, dtype: int64

In [35]:

amazon_2015_2025['order_date'].min(), amazon_2015_2025['order_date'].max()

(Timestamp('2015-01-01 00:00:00'), Timestamp('2025-12-31 00:00:00'))

In [36]:

amazon_2015_2025[amazon_2015_2025['order_date'] > pd.Timestamp.today()]

,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,is_festival_sale,festival_name,customer_rating,return_status,order_month,order_year,order_quarter,product_weight_kg,is_prime_eligible,product_rating
243,TXN_2025_00075626,2025-12-30,CUST_2024_00029815,PROD_001751,Realme Slate 4GB RAM Black,Electronics,Tablets,Realme,36360.21,26.79,...,False,No Festival,5.0,Delivered,12,2025,4,0.59,True,3.9
2843,TXN_2025_00069441,2025-12-31,CUST_2025_00004613,PROD_001293,OnePlus OnePlus Nord 4 64GB Black,Electronics,Smartphones,OnePlus,27964.78,27.27,...,False,No Festival,4.5,Delivered,12,2025,4,0.21,False,4.4
4379,TXN_2025_00072643,2025-12-30,CUST_2025_00011221,PROD_000943,Oppo Reno 6 64GB White,Electronics,Smartphones,Oppo,28092.04,0.00,...,False,No Festival,4.5,Delivered,12,2025,4,0.22,True,4.5
5057,TXN_2025_00068528,2025-12-30,CUST_2025_00000039,PROD_001904,Xiaomi Tracker Premium,Electronics,Smart Watch,Xiaomi,49950.70,0.00,...,False,No Festival,4.5,Delivered,12,2025,4,0.06,False,4.1
7016,TXN_2025_00075086,2025-12-31,CUST_2025_00021767,PROD_001410,Samsung Galaxy S25+ 128GB White,Electronics,Smartphones,Samsung,85537.89,0.00,...,False,No Festival,4.5,Delivered,12,2025,4,0.23,True,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120352,TXN_2025_00076560,2025-12-30,CUST_2020_00051940,PROD_001924,Fitbit Watch Deluxe,Electronics,Smart Watch,Fitbit,51082.59,0.00,...,False,No Festival,3.0,Delivered,12,2025,4,0.05,True,3.5
1125166,TXN_2025_00069365,2025-12-30,CUST_2024_00036729,PROD_001427,OnePlus OnePlus 13 256GB White,Electronics,Smartphones,OnePlus,52366.14,0.00,...,False,No Festival,4.0,Delivered,12,2025,4,0.20,True,4.6
1126253,TXN_2025_00072571,2025-12-30,CUST_2025_00014461,PROD_001465,Xiaomi Poco F7 64GB White,Electronics,Smartphones,Xiaomi,15126.92,29.83,...,False,No Festival,4.5,Delivered,12,2025,4,0.16,False,3.5
1127415,TXN_2025_00071337,2025-12-30,CUST_2017_00012194,PROD_001594,Acer Pavilion 8GB RAM Silver,Electronics,Laptops,Acer,206395.37,6.96,...,False,No Festival,5.0,Delivered,12,2025,4,2.46,False,3.4


In [37]:
# Subtotal logic
amazon_2015_2025['calc_subtotal'] = amazon_2015_2025['discounted_price_inr'] * amazon_2015_2025['quantity']

amazon_2015_2025['diff'] = abs(amazon_2015_2025['calc_subtotal'] - amazon_2015_2025['subtotal_inr'])

amazon_2015_2025[amazon_2015_2025['diff'] > 1]

,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,customer_rating,return_status,order_month,order_year,order_quarter,product_weight_kg,is_prime_eligible,product_rating,calc_subtotal,diff


In [38]:
amazon_raw = pd.read_csv("../data/amazon_india_complete_2015_2025.csv")

In [39]:
comparison = pd.DataFrame({
    'Before_Cleaning': amazon_raw.isna().sum(),
    'After_Cleaning': amazon_2015_2025.isna().sum()
})
comparison


,Before_Cleaning,After_Cleaning
brand,0,0.0
calc_subtotal,0,NaN
category,0,0.0
customer_age_group,0,0.0
customer_city,0,0.0
customer_id,0,0.0
customer_rating,0,0.0
customer_spending_tier,0,0.0
customer_state,0,0.0
customer_tier,0,0.0


In [40]:
amazon_india_2015_2025.isnull().sum()

transaction_id               0
order_date                   0
customer_id                  0
product_id                   0
product_name                 0
category                     0
subcategory                  0
brand                        0
original_price_inr           0
discount_percent             0
discounted_price_inr         0
quantity                     0
subtotal_inr                 0
delivery_charges             0
final_amount_inr             0
customer_city                0
customer_state               0
customer_tier                0
customer_spending_tier       0
customer_age_group           0
payment_method               0
delivery_days             6836
delivery_type                0
is_prime_member              0
is_festival_sale             0
festival_name                0
customer_rating              0
return_status                0
order_month                  0
order_year                   0
order_quarter                0
product_weight_kg            0
is_prime